# BoW and TF-IDF

In [130]:
import torch
import torchtext
import os
import collections

from torchtext.data.utils import ngrams_iterator

from torch.utils.data import DataLoader
import numpy as np

## Data

We will start with a simple text classification task based on AG_NEWS sample dataset, which is to classify news headlines into one of 4 categories: World, Sports, Business and Sci/Tech

In [91]:
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root="./data")
classes =  ['World', 'Sports', 'Business', 'Sci/Tech']

In [92]:
type(train_dataset)

torch.utils.data.datapipes.iter.sharding.ShardingFilterIterDataPipe

In [93]:
for i,x in zip(range(3),train_dataset):
    print(f"**{classes[x[0]]}** -> {x[1]}\n")

**Sci/Tech** -> Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.

**Sci/Tech** -> Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market.

**Sci/Tech** -> Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.



In [165]:
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
train_dataset = list(train_dataset)
test_dataset = list(test_dataset)

In [166]:
train_dataset[0]

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

In [167]:
# remove \\
dumi_train = []
dumi_test = []
for i in range(len(train_dataset)):
    dumi_train.append(list(train_dataset[i]))
for i in range(len(test_dataset)):
    dumi_test.append(list(test_dataset[i]))

for i in range(len(dumi_train)):
    dumi_train[i][1] = dumi_train[i][1].replace("\\"," ")
for i in range(len(dumi_test)):
    dumi_test[i][1] = dumi_test[i][1].replace("\\"," ")

for i in range(len(dumi_train)):
    dumi_train[i] = tuple(dumi_train[i])
for i in range(len(dumi_test)):
    dumi_test[i] = tuple(dumi_test[i])

In [168]:
train_dataset = dumi_train
test_dataset = dumi_test

## Tokenization and vectorization

* <code>torchtext.data.utils.get_tokenizer</code>
* <code>torchtext.vocab,vocab</code>
* <code>collection.Count</code>
* encode and decode with <code>vocab.get_stoi()[string]</code>

In [119]:
tokenizer = torchtext.data.utils.get_tokenizer("basic_english")

In [120]:
first_sentence = train_dataset[0][1]
f_tokens = tokenizer(first_sentence)
print(f'\nfirst token list:\n{f_tokens}')


first token list:
['wall', 'st', '.', 'bears', 'claw', 'back', 'into', 'the', 'black', '(', 'reuters', ')', 'reuters', '-', 'short-sellers', ',', 'wall', 'street', "'", 's', 'dwindling', 'band', 'of', 'ultra-cynics', ',', 'are', 'seeing', 'green', 'again', '.']


In [121]:
counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(tokenizer(line))
vocab = torchtext.vocab.vocab(counter, min_freq=1)
print("len of dicctionary : ", len(vocab))

len of dicctionary :  87235


In [122]:
word_lookup = [list((vocab[w], w)) for w in f_tokens]
print(f'\nIndex lockup in 1st sentence:\n{word_lookup}')


Index lockup in 1st sentence:
[[0, 'wall'], [1, 'st'], [2, '.'], [3, 'bears'], [4, 'claw'], [5, 'back'], [6, 'into'], [7, 'the'], [8, 'black'], [9, '('], [10, 'reuters'], [11, ')'], [10, 'reuters'], [12, '-'], [13, 'short-sellers'], [14, ','], [0, 'wall'], [15, 'street'], [16, "'"], [17, 's'], [18, 'dwindling'], [19, 'band'], [20, 'of'], [21, 'ultra-cynics'], [14, ','], [22, 'are'], [23, 'seeing'], [24, 'green'], [25, 'again'], [2, '.']]


In [123]:
def encode(x):
    return [vocab.get_stoi()[s] for s in tokenizer(x)]

vec = encode(first_sentence)
print(vec)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 10, 12, 13, 14, 0, 15, 16, 17, 18, 19, 20, 21, 14, 22, 23, 24, 25, 2]


In [124]:
def decode(x):
    return [vocab.get_itos()[i] for i in x]

decode(vec)

['wall',
 'st',
 '.',
 'bears',
 'claw',
 'back',
 'into',
 'the',
 'black',
 '(',
 'reuters',
 ')',
 'reuters',
 '-',
 'short-sellers',
 ',',
 'wall',
 'street',
 "'",
 's',
 'dwindling',
 'band',
 'of',
 'ultra-cynics',
 ',',
 'are',
 'seeing',
 'green',
 'again',
 '.']

## BiGrams, TriGrams and N-Grams

One limitation of word tokenization is that some words are part of multi word expressions, for example, the word _'hot dog'_ has a completely different meaning than the words 'hot' and 'dog' in other contexts. If we represent words 'hot` and 'dog' always by the same vectors, it can confuse the model.

To address this, **N-gram representations** are sometimes used in document classification, where the frequency of each word, bi-word or tri-word is a useful feature for training classifiers. 

To get n-gram representation, we can use `ngrams_iterator` function that will convert the sequence of tokens to the sequence of n-grams. In practice, n-gram vocabulary size is still too high to represent words as one-hot vectors, and thus we need to combine this representation with some dimensionality reduction techniques, such as **embeddings**.

In [126]:
bi_counter = collections.Counter()
for (label, line) in train_dataset:
    bi_counter.update(ngrams_iterator(tokenizer(line), 
                                      ngrams=2))

bi_vocab = torchtext.vocab.vocab(bi_counter, 
                                 min_freq=2)
print(f"Bigram vocab size = {len(bi_vocab)}")

Bigram vocab size = 505241


In [206]:
def encode(x):
    return [bi_vocab.get_stoi()[s] for s in tokenizer(x) if s in bi_vocab ]

encode(first_sentence)


[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 10,
 12,
 13,
 14,
 0,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 14,
 22,
 23,
 24,
 25,
 2]

## Bag of Words and TF-IDF

Bag of Words (BoW) vector representation is the most commonly used traditional vector representation. Each word is linked to a vector index, vector element contains the number of occurrences of a word in a given document.
> You can also think of BoW as a sum of all one-hot-encoded vectors for individual words in the text.

In [210]:
vocab_size = 10000
def to_bow(text,bow_vocab_size=vocab_size):#len(bi_vocab) 505241
    res = torch.zeros(bow_vocab_size,dtype=torch.float32)
    for i in encode(text):
        if i<bow_vocab_size:
            res[i] += 1
    return res

print(f"sample text:\n{train_dataset[0][1]}")
print(f"\nBoW vector:\n{to_bow(train_dataset[0][1])}")

sample text:
Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling band of ultra-cynics, are seeing green again.

BoW vector:
tensor([2., 1., 2.,  ..., 0., 0., 0.])


## Training BoW classifier

In [153]:
def bowify(b):
    return (
            torch.LongTensor([t[0]-1 for t in b]),
            torch.stack([to_bow(t[1]) for t in b])
    )

### DataLoader

In [157]:
print(train_dataset[0])
print(train_dataset[1])

(3, "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling band of ultra-cynics, are seeing green again.")
(3, 'Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group, which has a reputation for making well-timed and occasionally controversial plays in the defense industry, has quietly placed its bets on another part of the market.')


In [155]:
bowify([train_dataset[0],train_dataset[1]])

(tensor([2, 2]),
 tensor([[2., 1., 2.,  ..., 0., 0., 0.],
         [0., 0., 1.,  ..., 0., 0., 0.]]))

In [169]:
train_loader = DataLoader(train_dataset, batch_size=16, collate_fn=bowify, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, collate_fn=bowify, shuffle=True)

### Model

In [176]:
net = torch.nn.Sequential(torch.nn.Linear(vocab_size,4),
                          torch.nn.LogSoftmax(dim=1))

### Train

In [198]:
def train_epoch(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.NLLLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,features in dataloader:
        optimizer.zero_grad()
        out = net(features)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count

In [208]:
train_epoch(net,train_loader,epoch_size=1)

(0.08743439614772797, 0.3125)

## Term Frequency / Inverse Document Frequency  -  (TF-IDF)

In BoW representation, word occurrences are evenly weighted, regardless of the word itself. However, it is clear that frequent words, such as *'a'*, *'in'*, *'the'* etc. are much less important for the classification, than specialized terms. In fact, in most NLP tasks some words are more relevant than others.

**TF-IDF** stands for **term frequency–inverse document frequency**. It is a variation of bag of words, where instead of a binary 0/1 value indicating the appearance of a word in a document, a floating-point value is used, which is related to the **_frequency of word occurrence_** in the corpus.

The formula to calculate TF-IDF is:  $w_{ij} = tf_{ij}\times\log({N\over df_i})$

Here's the meaning of each parameter in the formula:
* $i$ is the word 
* $j$ is the document
* $w_{ij}$ is the weight or the importance of the word in the document
* $tf_{ij}$ is the number of occurrences of the word $i$ in the document $j$, i.e. the BoW value we have seen before
* $N$ is the number of documents in the collection
* $df_i$ is the number of documents containing the word $i$ in the whole collection

TF-IDF value $w_{ij}$ increases proportionally to the number of times a word appears in a document and is offset by the number of documents in the corpus that contains the word, which helps to adjust for the fact that some words appear more frequently than others. For example, if the word appears in *every* document in the collection, $df_i=N$, and $w_{ij}=0$, and those terms would be completely disregarded.

First, let's compute document frequency $df_i$ for each word $i$. We can represent it as tensor of size `vocab_size`. We will limit the number of documents to $N=1000$ to speed up processing. For each input sentence, we compute the set of words (represented by their numbers), and increase the corresponding counter:

In [212]:
N = 100
df = torch.zeros(vocab_size)
for _,line in train_dataset[:N]:
    for i in set(encode(line)):
        df[i] += 1

In [213]:
def tf_idf(s):
    bow = to_bow(s)
    return bow*torch.log((N+1)/(df+1))

print(tf_idf(train_dataset[0][1]))

tensor([7.0330, 3.5165, 0.1015,  ..., 0.0000, 0.0000, 0.0000])


> You may have noticed that we used a slightly different formula for TF-IDF, namely $\log({N+1\over df_i+1})$ instead of $\log({N\over df_i})$. This yields similar results, but prevents division by 0 in those cases when $df_i=0$.

Even though TF-IDF representation calculates different weights to different words according to their importance, it is unable to correctly capture the meaning, largely because the order of words in the sentence is still not taken into account. As the famous linguist J. R. Firth said in 1935, “The complete meaning of a word is always contextual, and no study of meaning apart from context can be taken seriously”. We will learn in the later units how to capture contextual information from text using language modeling.